### MicroVision 

#### Generating Semantic Log Template Embeddings

In [1]:
import os

if os.getcwd() == '/Users/matildamwendwa/Desktop/Desktop - Admin’s MacBook Pro/Python_Projects/microvision/notebooks':
    os.chdir('/Users/matildamwendwa/Desktop/Desktop - Admin’s MacBook Pro/Python_Projects/microvision')
    print("Changed!!")

print("Current working directory:", os.getcwd())

os.environ["TOKENIZERS_PARALLELISM"] = "false"

Changed!!
Current working directory: /Users/matildamwendwa/Desktop/Desktop - Admin’s MacBook Pro/Python_Projects/microvision


In [2]:
%pip install pandas numpy tqdm matplotlib sentence-transformers chromadb --quiet


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Loading the enriched Templates

In [3]:
import os
import pandas as pd

config = {
        "DATA_DIR": "data",
        "DATASET_NAME": "OpenStack",
        "DATASET_LOG": "_full.log",
        "ENRICHED_CSV": "_enriched.csv",
        "TEMPLATES_CSV": "_templates.csv",
        "TEMPLATES_JSON": "_templates.json",
        "PERSISTENCE_PATH": "persistence",
        "DRAIN_PATH": "drain3_state",
                
        "DEDUP_CSV": "_deduplicated.csv",
        "EMBEDDINGS_CSV": "_embeddings.csv",
        "CHROMA_PERSIST_DIR": "chroma_storage",
        "COLLECTION_NAME": "_collection"
}

os.makedirs(config["DATA_DIR"], exist_ok=True)

enriched_df = pd.read_csv(f"{config['DATA_DIR']}/{config['DATASET_NAME']}{config['DATASET_LOG']}{config['ENRICHED_CSV']}")
print(f"✅ Enriched data loaded successfully: {enriched_df.shape[0]} rows × {enriched_df.shape[1]} columns")
enriched_df.head(5)

✅ Enriched data loaded successfully: 207632 rows × 15 columns


,line_no,raw,content,template_id,template,File,Date,Time,Pid,Level,Component,Content,semantic_text,structured_metadata,Timestamp
0,1,nova-api.log.2017-05-14_21:27:04 2017-05-14 19...,[req-5a2050e7-b381-4ae9-92d2-8b08e9f9f4c0 113d...,1,[<*><REQ_ID>> <*><HASH>> <*><HASH>> - - -] <*>...,nova-api.log.2017-05-14_21:27:04,2017-05-14,19:39:01.445,25746,INFO,nova.osapi_compute.wsgi.server,[req-5a2050e7-b381-4ae9-92d2-8b08e9f9f4c0 113d...,[nova.osapi_compute.wsgi.server] [INFO] [<*><R...,{'component': 'nova.osapi_compute.wsgi.server'...,2017-05-14 19:39:01.445
1,2,nova-api.log.2017-05-14_21:27:04 2017-05-14 19...,[req-c26a7d54-55ab-412e-947f-421a2cb934fc 113d...,1,[<*><REQ_ID>> <*><HASH>> <*><HASH>> - - -] <*>...,nova-api.log.2017-05-14_21:27:04,2017-05-14,19:39:01.650,25746,INFO,nova.osapi_compute.wsgi.server,[req-c26a7d54-55ab-412e-947f-421a2cb934fc 113d...,[nova.osapi_compute.wsgi.server] [INFO] [<*><R...,{'component': 'nova.osapi_compute.wsgi.server'...,2017-05-14 19:39:01.650
2,3,nova-compute.log.2017-05-14_21:27:09 2017-05-1...,[req-e285b551-587f-4c1d-8eba-dceb2673637f 113d...,2,[<*><REQ_ID>> <*><HASH>> <*><HASH>> - - -] [in...,nova-compute.log.2017-05-14_21:27:09,2017-05-14,19:39:02.007,2931,INFO,nova.virt.libvirt.driver,[req-e285b551-587f-4c1d-8eba-dceb2673637f 113d...,[nova.virt.libvirt.driver] [INFO] [<*><REQ_ID>...,"{'component': 'nova.virt.libvirt.driver', 'lev...",2017-05-14 19:39:02.007
3,4,nova-api.log.2017-05-14_21:27:04 2017-05-14 19...,[req-eb681812-78ae-4a9f-9e2a-96e505285512 113d...,1,[<*><REQ_ID>> <*><HASH>> <*><HASH>> - - -] <*>...,nova-api.log.2017-05-14_21:27:04,2017-05-14,19:39:02.924,25746,INFO,nova.osapi_compute.wsgi.server,[req-eb681812-78ae-4a9f-9e2a-96e505285512 113d...,[nova.osapi_compute.wsgi.server] [INFO] [<*><R...,{'component': 'nova.osapi_compute.wsgi.server'...,2017-05-14 19:39:02.924
4,5,nova-compute.log.2017-05-14_21:27:09 2017-05-1...,[-] [instance: 2b590f10-49fd-4ec9-ae41-19596c2...,3,[-] [instance: <*>*> VM Stopped (Lifecycle Event),nova-compute.log.2017-05-14_21:27:09,2017-05-14,19:39:03.166,2931,INFO,nova.compute.manager,[-] [instance: 2b590f10-49fd-4ec9-ae41-19596c2...,[nova.compute.manager] [INFO] [-] [instance: <...,"{'component': 'nova.compute.manager', 'level':...",2017-05-14 19:39:03.166


### Step 2: Generate Semantic Embeddings

#### Sentence Transformer used: *** sentence-transformers/multi-qa-mpnet-base-dot-v1 ***

Initial Model used - ** all-MiniLM-L6-v2 **

In [4]:
from sentence_transformers import SentenceTransformer
import torch

device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

# model_name = "all-MiniLM-L6-v2"
# model_name = "multi-qa-mpnet-base-dot-v1"
model_name = "all-mpnet-base-v2"
model = SentenceTransformer(model_name, device=device)
print(f"✅ Loaded Sentence Transformer model: {model_name}, using device: {device}")

/Users/matildamwendwa/Desktop/Desktop - Admin’s MacBook Pro/Python_Projects/microvision/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✅ Loaded Sentence Transformer model: all-mpnet-base-v2, using device: mps


### ENCODING THE TEMPLATES

In [5]:

# Deduplication based on SHA1 hash
import hashlib
# Compute SHA1 hash for fast duplicate detection
enriched_df["hash"] = enriched_df["Content"].apply(lambda x: hashlib.sha1(x.encode('utf-8')).hexdigest())
hashed_df = enriched_df.drop_duplicates(subset="hash").drop(columns=["hash"]).reset_index(drop=True)

print(f"After Deduplication based on SHA1 hash: {len(hashed_df):,} unique log lines")


After Deduplication based on SHA1 hash: 151,354 unique log lines


In [6]:
import gc
import numpy as np
import torch
from tqdm import tqdm

def batch_encode_logs(df, text_column, batch_size, checkpoint_path):

    print("🔧 Generating embeddings for enriched templates...")

    # --- Generate embeddings ---
    logs = df[text_column].tolist()
    total_batches = int(np.ceil(len(logs) / batch_size))
    print(f"🚀 Starting encoding for {len(logs):,} log lines in {total_batches} batches...\n")

    # Load previous checkpoint if available
    start_idx = 0
    embeddings = []

    if os.path.exists(checkpoint_path):
        print(f"🔁 Found checkpoint: {checkpoint_path}. Resuming...")
        checkpoint = np.load(checkpoint_path, allow_pickle=True)
        embeddings = [torch.tensor(checkpoint["embeddings"])]
        start_idx = int(checkpoint["index"])
        print(f"Resuming from batch {start_idx:,}")

    # --- Batch encoding loop ---
    for batch_idx in tqdm(range(start_idx, len(logs), batch_size), desc="Encoding Logs in Batches", unit="batch"):

        batch_logs = logs[batch_idx: batch_idx + batch_size]

        with torch.no_grad():
            batch_embeddings = model.encode(batch_logs, device=device,
                convert_to_tensor=True,
                show_progress_bar=False,
                normalize_embeddings=True,

            )        
        embeddings.append(batch_embeddings.cpu())

         # Save checkpoint every N batches
        if (batch_idx // batch_size + 1) % 50 == 0 or (batch_idx + batch_size) >= total_batches:
            stacked = torch.cat(embeddings, dim=0).cpu().numpy()
            np.savez_compressed(checkpoint_path, embeddings=stacked, index=batch_idx + batch_size)
            # ---- CLEAN UP GPU
            embeddings = [torch.tensor(stacked)]  # keep single tensor to reduce many small tensors
            torch.mps.empty_cache()
            gc.collect()


    # STACK ALL BATCHES
    embeddings = torch.cat(embeddings, dim=0).numpy()
    np.savez_compressed(checkpoint_path.replace("_stream.npz", "_final.npz"), embeddings=embeddings)
    print(f"📦 Final embeddings saved to {checkpoint_path.replace('_stream.npz', '_final.npz')}")

    df["embedding"] = [emb.tolist() for emb in embeddings]

    df["embedding_id"] = [
        f"{r['Component']}_{r['template_id']}_{i}" if "template_id" in r else f"{r['Component']}_{i}"
        for i, r in df.iterrows()
    ]

    print(f"✅ Generated embeddings for {len(df):,} log lines")
    return df, embeddings


df, embeddings = batch_encode_logs(
    hashed_df, 
    text_column="semantic_text",
    batch_size=256, 
    checkpoint_path=f"{config['DATA_DIR']}/embeddings_stream",
)

# # --- Persist ---
EMBEDDINGS_CSV_PATH = f"{config['DATA_DIR']}/{config['DATASET_NAME']}{config['DATASET_LOG']}{config['EMBEDDINGS_CSV']}"
df.to_csv(EMBEDDINGS_CSV_PATH, index=False)
print(f"✅ Stored {len(df)} enriched templates with embeddings → {EMBEDDINGS_CSV_PATH}")

# # --- Preview ---
# print(enriched_df[["Component", "semantic_text", "structured_metadata", "Timestamp"]].head(3))


🔧 Generating embeddings for enriched templates...
🚀 Starting encoding for 151,354 log lines in 592 batches...



Encoding Logs in Batches: 100%|██████████| 592/592 [1:45:05<00:00, 10.65s/batch]    


📦 Final embeddings saved to data/embeddings_stream
✅ Generated embeddings for 151,354 log lines
✅ Stored 151354 enriched templates with embeddings → data/OpenStack_full.log_embeddings.csv


### Step 3: Store Embeddings in a Vector Database (ChromaDB)

#### 3.1 Initializing ChromaDB and some utility functions

In [ ]:
import ast
import json
import chromadb
from chromadb.config import Settings
from tqdm import tqdm
import os

# -------- Initializing ChromaDB Collection --------

def init_chroma_collection(
    persist_dir: str,
    collection_name: str,
    space: str = "cosine",
    verbose: bool = True,
):
    # Ensure directory exists
    os.makedirs(persist_dir, exist_ok=True)

    # Initialize persistent client
    client = chromadb.PersistentClient(path=persist_dir)

    # Retrieve or create collection
    collection = client.get_or_create_collection(
        name=collection_name,
        metadata={"hnsw:space": space}
    )

    if verbose:
        print(f"🔁 Loaded persistent Chroma from: {persist_dir}")
        print(f"🧩 Available collections: '{collection.name}' → Contains {collection.count()} embeddings")
        print("🧩 Existing collections:", [c.name for c in client.list_collections()])


    return client, collection

# -------- SAFE PARSING FUNCTION 
       
def safe_parse_embedding(x):
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        try:
            val = ast.literal_eval(x)
            if isinstance(val, list) and len(val) == 1 and isinstance(val[0], str):
                return ast.literal_eval(val[0])
            return val
        except Exception:
            return []
    return []

def safe_parse_metadata(x):
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        # Try JSON first (more robust)
        try:
            return json.loads(x)
        except Exception:
            # Try literal_eval fallback
            try:
                return ast.literal_eval(x)
            except Exception:
                return {"raw_metadata": x}  # fallback
    return {"raw_metadata": str(x)}

# -------- Repopulating ChromaDB from DataFrame --------

def repopulate_chroma_from_df(client, collection, df, batch_size):

    df["embedding"] = df["embedding"].apply(safe_parse_embedding)

    ids = df["embedding_id"].tolist()
    texts = df["semantic_text"].tolist()
    embeddings = df["embedding"].tolist()

    # Build metadata dictionary - prefer normalized metadata with ISO timestamps
    from src.enrichment.ingest_utils import df_to_metadatas

    if "structured_metadata" in df.columns:
        # parse existing structured metadata
        metadatas = df["structured_metadata"].apply(safe_parse_metadata).tolist()
        # if structured metadata lacks timestamp normalization but df has a timestamp column,
        # fall back to serializing rows to ensure consistent ISO-8601 timestamps
        if any(
            (not isinstance(m, dict) or all(k not in m for k in ("timestamp", "time", "ts", "created_at")))
            for m in metadatas
        ) and any(c in df.columns for c in ("timestamp", "time", "ts", "created_at")):
            meta_cols = [col for col in df.columns if col not in ["embedding", "semantic_text", "structured_metadata", "Timestamp"]]
            metadatas = df_to_metadatas(df, meta_cols=meta_cols)
    else:
        meta_cols = [col for col in df.columns if col not in ["embedding", "semantic_text", "structured_metadata", "Timestamp"]]
        metadatas = df_to_metadatas(df, meta_cols=meta_cols)

    print(f"✅ Inserting {len(texts)} semantic embeddings into ChromaDB (batch size={batch_size})")

    for i in tqdm(range(0, len(texts), batch_size), desc="🚀 Populating Chroma"):
        collection.add(
            documents=texts[i:i+batch_size],
            ids=ids[i:i+batch_size],
            embeddings=embeddings[i:i+batch_size],
            metadatas=metadatas[i:i+batch_size]
        )

    print(f"✅ Added {len(texts)} embeddings.")
    print(f"📦 Collection now contains {collection.count()} vectors.")
    # print("💾 Persistence handled automatically by Chroma ≥0.5.")


### 3.2 Run the Persistent Chroma Setup 

In [8]:
import re, ast, pandas as pd

chroma_persist_dir = f"{config['DATA_DIR']}/{config['PERSISTENCE_PATH']}/{config['CHROMA_PERSIST_DIR']}"
collection_name = f"{config['DATASET_NAME']}{config['DATASET_LOG']}{config['COLLECTION_NAME']}"

client, collection = init_chroma_collection(chroma_persist_dir, collection_name)


embeddings_df = pd.read_csv(EMBEDDINGS_CSV_PATH)
# data = np.load(f"{config['DATA_DIR']}/embeddings_stream.npz")
# embeddings = data["embeddings"]
# assert len(df) == len(embeddings), "Mismatch between DF and embeddings length"
# df["embedding"] = [emb.tolist() for emb in embeddings]

repopulate_chroma_from_df(client, collection, embeddings_df, batch_size=128)


🔁 Loaded persistent Chroma from: data/persistence/chroma_storage
🧩 Available collections: 'OpenStack_full.log_collection' → Contains 0 embeddings
🧩 Existing collections: ['OpenStack_full.log_collection']


: 

: 

##### Testing whether the Retrieval and Inference Stage works using sample tests

In [ ]:
sample_components = embeddings_df["Component"].unique()[:3]
print(f"🧩 Testing semantic retrieval for components: {list(sample_components)}")

for comp in tqdm(sample_components, desc="🔍 Validating retrieval per service"):
    query_row = embeddings_df[embeddings_df["Component"] == comp].sample(1).iloc[0]
    query_text = query_row["semantic_text"]
    query_embedding = query_row["embedding"]
    query_id = query_row.get("embedding_id", f"{query_row['Component']}_{query_row.get('template_id','0')}")

    print(f"\n🧠 Service: {comp}")
    print(f"📜 Query Template: {query_text}")

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5,
        include=["documents", "metadatas", "distances"]
    )

    print("\n🔎 Top 5 Semantic Matches:")
    for rank, (doc, meta, dist) in enumerate(
        zip(results["documents"][0], results["metadatas"][0], results["distances"][0]), start=1
    ):
        score = 1 - dist  # convert to similarity
        summary = f"  {rank}. [{meta.get('component', '?')}] → {doc[:80]}... (distance={dist:.4f}, sim={score:.4f})"
        if "status" in meta or "method" in meta:
            summary += f" [{meta.get('method','')}] [{meta.get('status','')}]"
        print(summary)
